# Banco de Colunas

### Objetivo
Definir banco de dados para os logs de acessos ao sistema ser armazenados de maneira otimizada para análises.

## Banco de Dados Escolhido
### Cassandra
O **Apache Cassandra** é um banco de dados NoSQL altamente escalável e distribuído, projetado para lidar com grandes volumes de dados em tempo real, com alta disponibilidade, tolerância a falhas e sem ponto único de falha.

### Exemplo de Utilização

In [1]:
# Instalando as bibliotecas necessárias
# %pip install cassandra-driver astrapy 

In [2]:
# Importando bibliotecas
import tomllib
import uuid
from datetime import datetime, UTC
from astrapy import DataAPIClient

### Segurança com TOML: 
Utilizei arquivos TOML para separar as credenciais sensíveis (.secrets.toml) das configurações de ambiente (settings.toml), simulando um ambiente produtivo real protegido por `.gitignore`.

In [3]:
# Carregando as configurações dos arquivos TOML
with open("config/settings.toml", "rb") as f:
    settings = tomllib.load(f)

with open("config/.secrets.toml", "rb") as f:
    secrets = tomllib.load(f)

### Exemplo

In [4]:
try:
    # Inicialização do Cliente
    client = DataAPIClient(secrets['astradb']['token'])
    db = client.get_database_by_api_endpoint(settings['astradb']['api_endpoint'])
    
    # Garantindo que a coleção exista (Evita o erro COLLECTION_NOT_EXIST)
    col_name = settings['astradb']['collection_name']
    if col_name not in db.list_collection_names():
        db.create_collection(col_name)
        print(f"Coleção '{col_name}' criada com sucesso!")

    collection = db.get_collection(col_name)

    # Registro de log com a sintaxe correta
    log_data = {
        "_id": str(uuid.uuid4()),
        "user_id": 12345,
        "timestamp": datetime.now(UTC).isoformat(), 
        "evento": "LOGIN_SUCCESS",
        "detalhes": {
            "browser": "Chrome",
            "device": "Desktop",
            "ip": "192.168.1.1"
        }
    }

    collection.insert_one(log_data)
    print(f"Sucesso: Evento registrado no Cassandra (Astra DB).")

except Exception as e:
    print(f"Erro ao processar operação: {e}")

Sucesso: Evento registrado no Cassandra (Astra DB).
